In [2]:
# Import the necessary dependencies.
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.stats as sts
import plotly.express as plt
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

**Create a null hypothesis, an alternative hypothesis, and choose a significance level. Use this cell to document your decisions.**

Question: Does having positive emotions 

Hypothesis: If disrupting your sleep is related to your stress levels the next day, waking up more than 10 times will result in higher stress levels the next day.

Null Hypothesis: Waking up more than 10 times wil result in no increase in the amount of stress the next day.

Alternative Hypothesis: Waking up more than 10 times wil result in an increase in the amount of stress the next day.

Significance Level:  Our p value for daily stress level is high for Daily stress levels between 10 - 30, sufficient evidence to NOT reject the null hypothesis.


Other notes:
There is a slight correlation between these factors:
- Daily Stress vs. Number of Awakenings
- Daily stress and Total amount of Sleep (TST) in minutes

In [3]:
# Get actvity DF for all users
sleep_list = []

for num in range(1,23):
    user_sleep_df = pd.read_csv(f'DataPaper/user_{num}/sleep.csv')
    stress_df = pd.read_csv(f'DataPaper/user_{num}/questionnaire.csv')

    user_sleep_df = user_sleep_df.merge(stress_df)
    sleep_list.append(user_sleep_df)
    
sleep_df = pd.concat(sleep_list)
sleep_df.head()

,Unnamed: 0,In Bed Date,In Bed Time,Out Bed Date,Out Bed Time,Onset Date,Onset Time,Latency,Efficiency,Total Minutes in Bed,...,panas_pos_10,panas_pos_14,panas_pos_18,panas_pos_22,panas_pos_9+1,panas_neg_10,panas_neg_14,panas_neg_18,panas_neg_22,panas_neg_9+1
0,0,2,00:46,2,03:31,2,00:46,0,87.27,165,...,21.0,17.0,12.0,18.0,17.0,11.0,13.0,13.0,10.0,10.0
0,0,2,00:50,2,06:22,2,00:54,4,73.49,332,...,37.0,32.0,24.0,27.0,33.0,11.0,10.0,16.0,17.0,18.0
0,0,1,22:29,1,05:52,1,22:32,3,79.23,443,...,35.0,34.0,31.0,28.0,35.0,11.0,12.0,11.0,12.0,11.0
0,0,2,00:57,2,07:10,2,01:01,4,85.52,373,...,30.0,27.0,22.0,19.0,26.0,11.0,13.0,15.0,14.0,14.0
0,0,1,23:56,1,06:42,1,23:56,0,85.71,406,...,30.0,25.0,31.0,27.0,31.0,26.0,17.0,17.0,15.0,16.0


In [6]:
# Sleeping less causes stress
sleep_df.columns

Index(['Unnamed: 0', 'In Bed Date', 'In Bed Time', 'Out Bed Date',
       'Out Bed Time', 'Onset Date', 'Onset Time', 'Latency', 'Efficiency',
       'Total Minutes in Bed', 'Total Sleep Time (TST)',
       'Wake After Sleep Onset (WASO)', 'Number of Awakenings',
       'Average Awakening Length', 'Movement Index', 'Fragmentation Index',
       'Sleep Fragmentation Index', 'MEQ', 'STAI1', 'STAI2', 'Pittsburgh',
       'Daily_stress', 'BISBAS_bis', 'BISBAS_reward', 'BISBAS_drive',
       'BISBAS_fun', 'panas_pos_10', 'panas_pos_14', 'panas_pos_18',
       'panas_pos_22', 'panas_pos_9+1', 'panas_neg_10', 'panas_neg_14',
       'panas_neg_18', 'panas_neg_22', 'panas_neg_9+1'],
      dtype='object')

In [7]:
# Correlation between Daily_stress and Total amount of Sleep (TST) in minutes
sleep_df.corr()

,Unnamed: 0,MEQ,STAI1,STAI2,Pittsburgh,Daily_stress,BISBAS_bis,BISBAS_reward,BISBAS_drive,BISBAS_fun,panas_pos_10,panas_pos_14,panas_pos_18,panas_pos_22,panas_pos_9+1,panas_neg_10,panas_neg_14,panas_neg_18,panas_neg_22,panas_neg_9+1
Unnamed: 0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MEQ,NaN,1.000000,0.064761,0.016279,-0.209774,0.126979,0.338104,-0.119249,-0.242214,-0.225220,0.437591,0.113905,0.182841,0.037962,0.198356,-0.111700,-0.328746,-0.298023,-0.484721,-0.336541
STAI1,NaN,0.064761,1.000000,0.320353,-0.038403,0.017660,0.296743,0.225850,0.262892,0.287548,-0.210540,-0.232667,0.150528,-0.058234,-0.172416,0.556821,0.503186,0.272083,-0.021982,-0.192108
STAI2,NaN,0.016279,0.320353,1.000000,-0.208806,-0.415194,0.384374,0.121036,0.147198,0.539860,0.029984,0.226447,0.348308,0.327436,0.134947,0.366462,0.174431,0.085288,0.041399,0.017200
Pittsburgh,NaN,-0.209774,-0.038403,-0.208806,1.000000,0.109530,-0.223407,0.148965,0.149766,0.061377,0.279767,0.441286,0.268345,0.479459,0.111870,0.032929,-0.109893,0.043959,0.027883,0.571732
Daily_stress,NaN,0.126979,0.017660,-0.415194,0.109530,1.000000,0.090844,0.105119,-0.203145,-0.184862,0.055891,-0.178304,0.048792,-0.184617,-0.354012,0.064892,-0.198414,-0.114323,-0.082023,0.009060
BISBAS_bis,NaN,0.338104,0.296743,0.384374,-0.223407,0.090844,1.000000,0.231605,-0.290883,0.044658,0.190807,-0.067705,0.102082,0.073580,0.211445,0.225481,-0.033812,0.273934,-0.223298,0.138724
BISBAS_reward,NaN,-0.119249,0.225850,0.121036,0.148965,0.105119,0.231605,1.000000,0.215584,0.383075,-0.077196,0.294002,0.350398,0.336434,0.197735,0.367624,0.214155,0.161039,-0.050582,0.110942
BISBAS_drive,NaN,-0.242214,0.262892,0.147198,0.149766,-0.203145,-0.290883,0.215584,1.000000,0.544991,-0.059940,0.212834,0.220702,0.117810,-0.101991,-0.118828,0.123986,-0.068761,0.078696,-0.112028
BISBAS_fun,NaN,-0.225220,0.287548,0.539860,0.061377,-0.184862,0.044658,0.383075,0.544991,1.000000,-0.226014,0.112428,0.188805,0.113224,-0.192862,0.179195,0.216289,0.315479,0.041451,0.056343


In [9]:
# Scatter plot showing the quantity of sleep and stress level
plt = px.scatter(sleep_df, x='panas_pos_9+1', y='Pittsburgh', color='panas_pos_22')
plt.show()
